# Introduction to Markov Chain Monte-Carlo Methods

## Rohan L. Fernando

## February 2016



-   Gibbs sampler is widely used for drawing samples from some distribution

Gibbs Sampler
-------------

-   Want to draw samples from $f(x_{1},x_{2},\ldots,x_{n})$

-   Even though it may be possible to compute
    $f(x_{1},x_{2},\ldots,x_{n})$, it is often difficult to draw samples
    directly from $f(x_{1},x_{2},\ldots,x_{n})$

-   Gibbs:

    -   Get valid a starting point $\mathbf{x}^{0}$

    -   Draw sample $\mathbf{x}^{t}$ as:
        $$\begin{matrix}x_{1}^{t} & \text{from} & f(x_{1}|x_{2}^{t-1},x_{3}^{t-1},\ldots,x_{n}^{t-1})\\
        x_{2}^{t} & \text{from} & f(x_{2}|x_{1}^{t},x_{3}^{t-1},\ldots,x_{n}^{t-1})\\
        x_{3}^{t} & \text{from} & f(x_{3}|x_{1}^{t},x_{2}^{t},\ldots,x_{n}^{t-1})\\
        \vdots &  & \vdots\\
        x_{n}^{t} & \text{from} & f(x_{n}|x_{1}^{t},x_{2}^{t},\ldots,x_{n-1}^{t})
        \end{matrix}$$

-   The sequence
    ${\mathbf x}^{1},{\mathbf x}^{2},\ldots,{\mathbf x}^{n}$
    is a Markov chain with stationary distribution
    $f(x_{1},x_{2},\ldots,x_{n})$

Making Inferences from Markov Chain
-----------------------------------

Can show that samples obtained from a <font color='red'>Markov chain</font> can be
used to draw inferences from $f(x_{1},x_{2},\ldots,x_{n})$ provided the
chain is:

-   <font color='red'>Irreducible</font>: can move from any state $i$ to any other
    state $j$

-   <font color='red'>Positive recurrent</font>: return time to any state has finite
    expectation

-   *Markov Chains*, J. R. Norris (1997)

##  Bivariate Normal Example

Let $f(\mathbf{x})$ be a bivariate normal density with
  means
$$
\mu' =
\begin{bmatrix}
  1 & 2
\end{bmatrix}
$$
and covariance matrix
$$
\mathbf{V} =
\begin{bmatrix}
  1 & 0.5\\
0.5& 2.0
\end{bmatrix}
$$


Suppose we do not know how to draw samples from $f(\mathbf{x})$, but know how
to draw samples from $f(x_i|x_j)$, which is univariate normal with mean:
$$
\mu_{i.j} = \mu_i + \frac{v_{ij}}{v_{jj}}(x_j - \mu_j)
$$
and variance
$$
v_{i.j} = v_{ii} - \frac{v^2_{ij}}{v_{jj}}
$$

In [2]:
using Distributions
nSamples = 20000
m = [1.0; 2.0]
v = [1.0 0.5; 0.5 2.0]
y   = fill(0.0,2)
save = fill(0.0,2,nSamples)
sum = fill(0.0,2)
s12 = sqrt(v[1,1] - v[1,2]*v[1,2]/v[2,2])
s21 = sqrt(v[2,2] -  v[1,2]*v[1,2]/v[1,1])
m1 = 0
m2 = 0;
for (iter in 1:nSamples)
    m12 = m[1] + v[1,2]/v[2,2]*(y[2] - m[2])
    y[1] = rand(Normal(m12,s12),1)[1]
    m21 = m[2] + v[1,2]/v[1,1]*(y[1] - m[1])
    y[2] = rand(Normal(m21,s21),1)[1]
    sum += y
    save[:,iter] = y 
    if iter%1000 == 0 
        mean = sum/iter
        @printf "%10d %8.2f %8.2f \n" iter mean[1]  mean[2]
    end
end

      1000     0.98     1.93 
      2000     0.99     1.96 
      3000     0.98     1.95 
      4000     0.99     1.95 
      5000     0.98     1.95 
      6000     0.99     1.96 
      7000     0.98     1.96 
      8000     0.98     1.96 
      9000     0.98     1.97 
     10000     0.98     1.97 
     11000     0.98     1.97 
     12000     0.98     1.98 
     13000     0.98     1.98 
     14000     0.98     1.98 
     15000     0.98     1.98 
     16000     0.98     1.98 
     17000     0.98     1.98 
     18000     0.98     1.98 
     19000     0.98     1.99 
     20000     0.98     1.98 


In [3]:
cov(save',)

2x2 Array{Float64,2}:
 0.988526  0.496471
 0.496471  2.00116 

## Metropolis-Hastings Algorithm

* Sometimes may not be able to draw samples directly from $f(x_i|\mathbf{x}_{i\_})$ 

* Convergence of the Gibbs sampler may be too slow

* Metropolis-Hastings (MH) for sampling from $f(\mathbf{x})$: 

	

* a candidate sample, $y$, is drawn from a proposal distribution $q(y|x^{t-1})$

	$$
	x^t =  \begin{cases}
	           y            &  \text{with probability}\, \alpha \\
               x^{t-1}     & \text{with probability}\, 1 - \alpha \\ 
		   \end{cases}
	$$
	
$$ \alpha = \min(1,\frac{f(y)q(x^{t-1}|y)}{f(x^{t-1})q(y|x^{t-1})}) $$
 
    
* The samples from MH is a Markov chain with stationary distribution $f(x)$      

## Bivariate Normal Example

In [5]:
nSamples = 100000
m = [1.0, 2.0]
v = [1.0 0.5; 0.5 2.0]
vi = inv(v)
y   = fill(0.0,2)
sum = fill(0.0,2)

m1 = 0
m2 = 0
xx = 0
y1 = 0
delta = 1.0
min1 = -delta*sqrt(v[1,1])
max1 = +delta*sqrt(v[1,1])
min2 = -delta*sqrt(v[2,2])
max2 = +delta*sqrt(v[2,2])
z = y-m
denOld = exp(-0.5*z'*vi*z)
d1 = Uniform(min1,max1)
d2 = Uniform(min2,max2)
ynew = fill(0.0,2);
for (iter in 1:nSamples)
    ynew[1] = y[1] + rand(d1,1)[1]
    ynew[2] = y[2] + rand(d2,1)[1]
   denNew = exp(-0.5*(ynew-m)'*vi*(ynew-m));
   alpha = denNew/denOld;
    u = rand()
    if (u < alpha[1]) 
        y = copy(ynew)
        denOld = exp(-0.5*(y-m)'*vi*(y-m)) 
    end
    sum += y
    mean = sum/iter
    if iter%1000 == 0 
        @printf "%10d %8.2f %8.2f \n" iter mean[1]  mean[2]
    end
end

      1000     1.14     2.39 
      2000     1.14     2.08 
      3000     1.07     1.96 
      4000     1.01     1.91 
      5000     1.00     1.92 
      6000     0.99     1.91 
      7000     1.01     1.90 
      8000     1.00     1.86 
      9000     0.99     1.84 
     10000     0.99     1.85 
     11000     1.01     1.88 
     12000     1.01     1.88 
     13000     1.04     1.95 
     14000     1.05     1.96 
     15000     1.04     1.97 
     16000     1.04     1.98 
     17000     1.03     1.97 
     18000     1.04     1.99 
     19000     1.03     1.99 
     20000     1.02     1.96 
     21000     1.01     1.96 
     22000     1.02     1.95 
     23000     1.01     1.94 
     24000     1.00     1.96 
     25000     1.01     1.96 
     26000     1.02     1.99 
     27000     1.01     1.98 
     28000     1.01     1.99 
     29000     1.01     1.99 
     30000     1.02     2.00 
     31000     1.03     2.00 
     32000     1.02     2.00 
     33000     1.03     2.01 
     34000

In [40]:
randn(2,3)

2x3 Array{Float64,2}:
 -0.110193  -0.714521   1.93687  
 -0.936767   1.69849   -0.0425868